In [ ]:
# Retrieve the band, model name, RAM capacity, market regions, and the date when the information was added for device(s) associated with a specified code name.
# Open the CSV file and use 'with' to ensure it's properly closed after reading
with open(file_location, "r") as file:
    data = csv.reader(file)
    code_name = input("Enter the code name: ")

    # Create a flag to check if the code name is found
    code_name_found = False

    # Loop through each row in the CSV file
    for row in data:
        if row and row[7] == code_name:
            # If the code name is found, print the details
            value1 = row[41]
            value2 = row[2]
            value3 = row[23]
            value4 = row[44]
            value5 = row[45]

            print("The Code name {} details are:".format(code_name))
            print("The band(WLAN) of the device is:", value1)
            print("The model name is:", value2)
            print("The Ram of the devics is:", value3)
            print("The market region is:", value4)
            print("The date the information was added is:", value5)

            code_name_found = True
            break  # Exit the loop as we found the ID

    if not code_name_found:
        print("No match for the OEM ID details found.")   

In [3]:
import csv
# "device_features.csv"
file_location = input("Enter location: ")
# Assuming you have a list of lists called 'data'
with open(file_location, "r") as file:
    data = csv.reader(file)
    result_rows = []
    code_name = input("Enter the code name: ")

    # Create a flag to check if the code name is found
    code_name_found = False

    # Loop through each row in the CSV file
    for row in data:
        if row and row[23] == code_name:
            # If the code name is found, print the details
            value1 = row[41]
            value2 = row[2]
            value3 = row[23]
            value4 = row[44]
            value5 = row[45]
            row_dict = {
                "Band(WLAN)": value1,
                "Model name": value2,
                "RAM": value3,
                "Market regions": value4,
                "Date of addition": value5
            }
            result_rows.append(row_dict)

            code_name_found = True

    if not code_name_found:
        print("No match for the OEM ID details found.")   

    else:
        num_rows = 20
        start_pos = 0
        end_pos = min(start_pos + num_rows, len(result_rows))

        while True:
            for i in range(start_pos, end_pos):
                row = result_rows[i]
                print("Band(WLAN):", row["Band(WLAN)"])
                print("Model name:", row["Model name"])
                print("RAM:", row["RAM"])
                print("Market region:", row["Market regions"])
                print("Date of addition:", row["Date of addition"])
                print()

            if end_pos >= len(result_rows):
                print("No more rows available.")
                break

            user_input = input("Enter 'next' to retrieve the next 20 rows, or 'quit' to exit: ")
            if user_input == 'next':
                start_pos += num_rows
                end_pos = min(start_pos + num_rows, len(result_rows))
            elif user_input == 'quit':
                break


Enter location:  device_features.csv
Enter the code name:  3


Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: SM-A135U Galaxy A13 2022 Standard Edition TD-LTE US 32GB / SM-A135R4
RAM: 3
Market region: North America
Date of addition: 02-08-23 18:10

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: SM-A135U1/DS Galaxy A13 2022 Standard Edition Dual SIM TD-LTE US 32GB
RAM: 3
Market region: North America
Date of addition: 01-08-23 09:00

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: SM-A135U Galaxy A13 2022 Standard Edition TD-LTE US 32GB / SM-A135V
RAM: 3
Market region: North America
Date of addition: 31-07-23 23:26

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: SM-A135U Galaxy A13 2022 Standard Edition Dual SIM TD-LTE US 32GB / SM-A135T
RAM: 3
Market region: North America
Date of addition: 21-07-23 22:38

Band(WLAN): 802.11b,802.11g,802.11n
Model name: C110 2023 TD-LTE US 32GB N156DL
RAM: 3
Market region: North America
Date of addition: 29-06-23 14:25

Band(WLAN): 802.11b,802.11

Enter 'next' to retrieve the next 20 rows, or 'quit' to exit:  next


Band(WLAN): 802.11b,802.11g,802.11n
Model name: G11 2022 Global Dual SIM TD-LTE 32GB
RAM: 3
Market region: Asia,Eastern Europe,Europe,North America,Western Europe
Date of addition: 18-01-23 17:10

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: iPad 10.2-inch 2021 9th gen A2602 WiFi 64GB
RAM: 3
Market region: Africa,Asia,Australia,Central America,Eastern Europe,Europe,Middle East,North America,Oceania,South America,Southeast Asia,Worldwide
Date of addition: 25-11-21 14:36

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: iPad 10.2-inch 2021 9th gen A2602 WiFi 256GB
RAM: 3
Market region: Africa,Asia,Australia,Central America,Eastern Europe,Europe,Middle East,North America,Oceania,South America,Southeast Asia,Worldwide
Date of addition: 25-11-21 14:21

Band(WLAN): 802.11a,802.11b,802.11g,802.11n,802.11ac
Model name: iPad 10.2-inch 2021 9th gen A2603 TD-LTE NA 64GB
RAM: 3
Market region: North America
Date of addition: 25-11-21 14:07

Band(WLAN): 802.11a,80

Enter 'next' to retrieve the next 20 rows, or 'quit' to exit:  quit


In [ ]:
# "device_features.csv"
file_location = input("Enter location: ")
with open(file_location, "r") as file:
    csv_reader = csv.reader(file)
    # for row in csv_reader:
        # print(row)
# a1. Retrieve the model name, manufacturer, weight, price, and price unit for the device(s) based on the oem_id.
# Loop through each row in the CSV file
result_rows = []
while True:
    oem_id = input("Enter the oem id: ")
    for row in file:
        # Convert row[0] to integer for comparison
        if row[0] == oem_id:
            value1 = row[2]
            value2 = row[6]
            value3 = row[14]
            value4 = row[15]
            value5 = row[16]
            # If it is, add the entire row to the list of matching rows
            result_rows.append(row)
            # Print the matching rows
    if result_rows:
        print("The Oem Id number {} details are :\n".format(oem_id))
        print("The model of the device is :", value1)
        print("The manufactuerer is :", value2)
        print("The weight of the device is :", value3)
        print("The price of the device is : ", value4)
        print("The price unit is : ", value5)
    else:
        print("No match for the oem Id details found.")
return